
# RAG con PDFs (Notebook paralelo)
Este cuaderno replica el flujo de `rag_cli.py` pero muestra cada etapa con resúmenes y estadísticas para entender los datos procesados.


In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from rag_modulo3 import (
    PDF_DIR,
    COLLECTION_NAME,
)
from rag_modulo3.preparation import (
    load_pdf_documents,
    extract_metadata,
    chunk_documents,
    build_vector_store,
)
from rag_modulo3.rag_chain import (
    build_query_rewriter,
    answer_question,
)

load_dotenv()
print(f"📁 Directorio de PDFs: {PDF_DIR.resolve()}")
print(f"📚 Colección en uso: {COLLECTION_NAME}")


📁 Directorio de PDFs: /Users/fabiansalfatenavarro/Desktop/Diplomando IA/Archivos/M3/Tarea Grupal/pdf
📚 Colección en uso: rag_mod3_pdf_exportaciones


## Inventario de PDFs

In [14]:

pdf_records = []
for path in sorted(PDF_DIR.glob('*.pdf')):
    stat = path.stat()
    pdf_records.append({
        "archivo": path.name,
        "tam_mb": round(stat.st_size / 1e6, 2),
        "modificado": stat.st_mtime,
    })

pdf_df = pd.DataFrame(pdf_records)
if pdf_df.empty:
    raise RuntimeError("No hay PDFs en la carpeta pdf/.")
pdf_df


,archivo,tam_mb,modificado
0,Claves-para-hacer-negocios-con-Singapur-2025 v...,7.58,1.766718e+09
1,Claves_para_hacer_negocios_Emiratos-Arabes-Uni...,7.36,1.766718e+09
2,Claves_para_hacer_negocios_Espana_2025 v2.pdf,9.01,1.766718e+09
3,Estudio-Claves-para-hacer-negocios-con-Japon-2...,5.43,1.766718e+09
4,NoCobre_NoLitio_noviembre-1 v2.pdf,5.82,1.766718e+09


## 1. Carga completa de documentos

In [15]:
raw_documents = load_pdf_documents()
print(f"Documentos cargados: {len(raw_documents)}")
print("\nEjemplo de metadata base:")
print(raw_documents[0].metadata)
print("\nFragmento del contenido:\n", raw_documents[0].page_content[:500])


Documentos cargados: 5

Ejemplo de metadata base:
{'source': 'pdf/Claves-para-hacer-negocios-con-Singapur-2025 v2.pdf'}

Fragmento del contenido:
 3¡ 
ProChile” ssssssssssssssssssssssssssssssss 
*x HACIENDO 
NEGOCIOS CON EL MUNDO 
AÑOS ¢ 
Claves para 
hacer negocios con 
Singapur| 2025 
Oficina Comercial de ProChile en Kuala Lumpur, Malasia 
ProChile - Todos los derechos reservados. 
Marzo, 2025

Claves para hacer negocios con Singapur| 2025 
1. Singapur como mercado 
La isla de Singapur se encuentra a 137 Km al norte del 
ecuador. Está formado por 64 islas incluyendo la isla 
principal conocida como la isla de Singapur o Pulau 
Ujong. 
La


## 2. Extracción de metadata con GPT-4o

In [16]:

llm = ChatOpenAI(model='gpt-4o', temperature=0.1)
metadata_map = extract_metadata(raw_documents, llm)
metadata_df = pd.DataFrame([
    {
        'source': src,
        'titulo': meta.titulo,
        'categoria': meta.categoria,
        'resumen': meta.resumen,
    }
    for src, meta in metadata_map.items()
])
metadata_df


,source,titulo,categoria,resumen
0,pdf/Claves-para-hacer-negocios-con-Singapur-20...,Guía de Negocios en Singapur 2025,Negocios Internacionales,Singapur es un centro financiero y comercial e...
1,pdf/Claves_para_hacer_negocios_Emiratos-Arabes...,Claves para el Éxito Empresarial en EAU,Negocios Internacionales,Los empresarios en Emiratos Árabes Unidos dest...
2,pdf/Claves_para_hacer_negocios_Espana_2025 v2.pdf,Claves para Hacer Negocios con España en 2025,Negocios Internacionales,El documento explora las tendencias clave del ...
3,pdf/Estudio-Claves-para-hacer-negocios-con-Jap...,Claves para Negocios con Japón,Negocios Internacionales,El documento destaca la prudencia del empresar...
4,pdf/NoCobre_NoLitio_noviembre-1 v2.pdf,Análisis de Exportaciones Chilenas 2025,Economía y Comercio Exterior,El informe de ProChile analiza las exportacion...


## 3. Semantic chunking y estadísticas

In [17]:
chunk_embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
chunker = SemanticChunker(
    chunk_embeddings,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=50,
)
chunked_docs = chunk_documents(raw_documents, metadata_map, chunker)
print(f"Chunks generados: {len(chunked_docs)}")

stats_df = pd.DataFrame([
    {
        'titulo': doc.metadata.get('titulo', 'N/A'),
        'source': doc.metadata.get('source'),
        'chars': len(doc.page_content),
        'words': len(doc.page_content.split()),
    }
    for doc in chunked_docs
])
print("\nResumen por documento:")
stats_summary = stats_df.groupby(['titulo']).agg(
    chunks=('source', 'count'),
    palabras_promedio=('words', 'mean'),
)
stats_summary


Chunks generados: 440

Resumen por documento:


,chunks,palabras_promedio
titulo,,
Análisis de Exportaciones Chilenas 2025,37,81.972973
Claves para Hacer Negocios con España en 2025,146,40.993151
Claves para Negocios con Japón,76,56.276316
Claves para el Éxito Empresarial en EAU,113,50.548673
Guía de Negocios en Singapur 2025,68,47.941176


## 4. Creación / actualización del vector store en Qdrant

In [8]:

vector_store = build_vector_store(chunked_docs)
print('Vector store actualizado en Qdrant. Colección:', vector_store.collection_name)


Vector store actualizado en Qdrant. Colección: rag_mod3_pdf_exportaciones


### Estado de la colección en Qdrant

In [9]:
from rag_modulo3 import COLLECTION_NAME, get_qdrant_client

client = get_qdrant_client()
collection = client.get_collection(COLLECTION_NAME)
params = collection.config.params
vector_size = None
if hasattr(params, "vectors") and isinstance(params.vectors, dict):
    default_vec = params.vectors.get("")
    if default_vec is not None:
        vector_size = getattr(default_vec, "size", None)
elif isinstance(params, dict):
    vectors = params.get("vectors", {})
    if isinstance(vectors, dict):
        default_vec = vectors.get("")
        if isinstance(default_vec, dict):
            vector_size = default_vec.get("size")

print(f"Colección analizada: {COLLECTION_NAME}")
print("Vectores indexados:", collection.points_count)
print("Dimensiones:", vector_size)


Colección analizada: rag_mod3_pdf_exportaciones
Vectores indexados: 718
Dimensiones: 3072


## 5. Consulta de ejemplo con reescritura

In [11]:
# Después de crear el vector_store:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

rewrite_chain = build_query_rewriter(llm)
demo_query = "¿Qué oportunidades comerciales hay para exportar servicios en 2025?"
response = answer_question(demo_query, retriever, llm, rewrite_chain)
print(response)


En 2025, las oportunidades para exportar servicios desde Chile se presentan principalmente en sectores que han mostrado un crecimiento significativo. Según el "Análisis de Exportaciones Chilenas 2025", los servicios de Tecnologías de la Información y Comunicación (TICs) lideran con un valor FOB de US$ 909 millones y un crecimiento del 2,7%. Otros sectores destacados incluyen los servicios financieros, con un crecimiento del 4,4%, y los servicios de administración, que crecieron un 3,5% (Fuente: Análisis de Exportaciones Chilenas 2025).

Además, el "Manual de Exportación de Servicios" sugiere que la diversificación de la oferta exportable hacia servicios de alto valor agregado es una estrategia viable para integrarse en cadenas globales de valor. Esto implica que sectores como la investigación y desarrollo, la asesoría, y los servicios logísticos también presentan oportunidades significativas para la exportación (Fuente: Guía para la Exportación de Servicios).

En términos de mercados, 

In [3]:
from rag_modulo3.config import load_environment, COLLECTION_NAME
from rag_modulo3.rag_chain import load_vector_store
from langchain_openai import OpenAIEmbeddings

load_environment()
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = load_vector_store(embeddings)

def max_score(query):
    results = vector_store.similarity_search_with_relevance_scores(query, k=3)
    return [(doc.metadata.get("titulo"), score) for doc, score in results]

print(max_score("Información de España"))
print(max_score("Información de Chile"))
print(max_score("Constitución de Estados Unidos"))
print(max_score("Información de Japon"))
print(max_score("Información de PlayStation 5"))


[('Claves para Hacer Negocios con España en 2025', 0.754091025), ('Claves para Hacer Negocios con España en 2025', 0.74984123), ('Claves para Hacer Negocios con España en 2025', 0.74843865)]
[('Claves para el Éxito Empresarial en EAU', 0.78993702), ('Análisis de Exportaciones Chilenas sin Cobre ni Litio', 0.77982365), ('Guía de Negocios en Singapur 2025', 0.77752585)]
[('Claves para el Éxito Empresarial en EAU', 0.650950285), ('Claves para Hacer Negocios con España en 2025', 0.6445491), ('Claves para el Éxito Empresarial en EAU', 0.64258033)]
[('Claves para Negocios con Japón', 0.7773887500000001), ('Claves para Negocios con Japón', 0.756977765), ('Claves para Negocios con Japón', 0.74979019)]
[('Claves para Negocios con Japón', 0.6643162499999999), ('Claves para Hacer Negocios con España en 2025', 0.6566413799999999), ('Claves para Hacer Negocios con España en 2025', 0.6503548)]
